# BeforeIT Model: Run, Plot, and Tweak Params

This notebook demonstrates how to:
1. **Run the model** with default parameters
2. **Plot key economic variables** from the simulation
3. **Test sensitivity** by adjusting parameters

Let's walk through each step!

## 1. Setup and Initialize the Model

First, let's import the necessary packages and initialize our model wrapper.

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from uk_abm.julia_model_wrapper import BeforeITModelWrapper

# Set up nice plotting defaults for matplotlib (still used for some legacy plots)
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10



In [26]:
# Initialize the BeforeIT model with Austrian 2010Q1 parameters
model = BeforeITModelWrapper(
    base_parameters="AUSTRIA2010Q1", 
    model_path="../dev/BeforeIT.jl"
)


INFO:uk_abm.julia_model_wrapper:Setting up Julia environment...


INFO:uk_abm.julia_model_wrapper:Activating BeforeIT environment at: ../dev/BeforeIT.jl
  Activating project at `~/Documents/00. Bank of England/03. MPIL/uk_abm/dev/BeforeIT.jl`
INFO:uk_abm.julia_model_wrapper:Installing package dependencies...
INFO:uk_abm.julia_model_wrapper:Importing BeforeIT and required packages...
INFO:uk_abm.julia_model_wrapper:BeforeIT AUSTRIA2010Q1 parameters loaded successfully
INFO:uk_abm.julia_model_wrapper:Julia environment setup completed successfully
INFO:uk_abm.julia_model_wrapper:Found 44 calibratable parameters: ['tau_SIW', 'tau_INC', 'theta_UB', 'tau_SIF', 'T_max', 'beta_pi_EA', 'tau_G', 'psi', 'alpha_G', 'r_star', 'zeta_LTV', 'beta_Y_EA', 'tau_VAT', 'alpha_Y_EA', 'beta_E', 'rho', 'S', 'r_G', 'alpha_E', 'T', 'L', 'sigma_E', 'beta_I', 'theta', 'alpha_I', 'sigma_pi_EA', 'H_inact', 'zeta', 'tau_EXPORT', 'pi_star', 'sigma_Y_EA', 'T_prime', 'beta_G', 'zeta_b', 'mu', 'psi_H', 'sigma_G', 'sigma_I', 'G', 'tau_FIRM', 'J', 'theta_DIV', 'xi_gamma', 'tau_CF']


In [27]:
# Let's see what parameters we can calibrate and what variables we can analyze
param_names = model.get_parameter_names()
current_values = model.get_current_parameter_values()

for name in param_names[:10]:  # Show first 10 parameters
    print(f"  • {name:15s}: {current_values[name]:.4f}")

if len(param_names) > 10:
    print(f"  ... and {len(param_names) - 10} more parameters")

print("\nAvailable output variables:")
variable_names = model.get_variable_names()
for name in variable_names[:10]:  # Show first 10 variables
    print(f"  • {name}")
if len(variable_names) > 10:
    print(f"  ... and {len(variable_names) - 10} more variables")


  • tau_SIW        : 0.1711
  • tau_INC        : 0.2134
  • theta_UB       : 0.3586
  • tau_SIF        : 0.2122
  • T_max          : 12.0000
  • beta_pi_EA     : 0.0026
  • tau_G          : 0.0091
  • psi            : 0.9097
  • alpha_G        : 0.9906
  • r_star         : -0.0034
  ... and 34 more parameters

Available output variables:
  • collection_time
  • nominal_gdp
  • real_gdp
  • nominal_gva
  • real_gva
  • nominal_household_consumption
  • real_household_consumption
  • nominal_government_consumption
  • real_government_consumption
  • nominal_capitalformation
  ... and 18 more variables


## 2. Run the Model

Now let's run a simulation with the default parameters to see how the economy evolves over time.

In [28]:
# Run simulation with default parameters
# Use the current parameter values as our baseline
baseline_params = list(current_values.values())

# Run simulation for 40 quarters (10 years)
T = 40
seed = 42

results = model.run_simulation(
    parameters=baseline_params,
    T=T,
    seed=seed,
    ensemble_size=4  # Average over 4 runs
)
# print(f"Simulation completed Output shape: {results.original_shape}")
# print(f"   Time periods: {T}, Variables: {results.shape[1]}")

# Create time axis (quarters)
time_quarters = np.arange(1, T + 1)


INFO:uk_abm.julia_model_wrapper:Running simulation with T=40, ensemble_size=4, seed=42
INFO:uk_abm.julia_model_wrapper:Simulation completed successfully


In [29]:
results.get_scalar_timeseries().keys()


dict_keys(['nominal_gdp', 'real_gdp', 'nominal_gva', 'real_gva', 'nominal_household_consumption', 'real_household_consumption', 'nominal_government_consumption', 'real_government_consumption', 'nominal_capitalformation', 'real_capitalformation', 'nominal_fixed_capitalformation', 'real_fixed_capitalformation', 'nominal_fixed_capitalformation_dwellings', 'real_fixed_capitalformation_dwellings', 'nominal_exports', 'real_exports', 'nominal_imports', 'real_imports', 'operating_surplus', 'compensation_employees', 'wages', 'taxes_production', 'gdp_deflator_growth_ea', 'real_gdp_ea', 'euribor'])

## 3. Plot Key Economic Indicators

Let's visualize the key economic variables from our simulation.


In [30]:
print(variable_names)


['collection_time', 'nominal_gdp', 'real_gdp', 'nominal_gva', 'real_gva', 'nominal_household_consumption', 'real_household_consumption', 'nominal_government_consumption', 'real_government_consumption', 'nominal_capitalformation', 'real_capitalformation', 'nominal_fixed_capitalformation', 'real_fixed_capitalformation', 'nominal_fixed_capitalformation_dwellings', 'real_fixed_capitalformation_dwellings', 'nominal_exports', 'real_exports', 'nominal_imports', 'real_imports', 'operating_surplus', 'compensation_employees', 'wages', 'taxes_production', 'gdp_deflator_growth_ea', 'real_gdp_ea', 'euribor', 'nominal_sector_gva', 'real_sector_gva']


In [31]:
selected_variables = [
    'real_gdp',
    'real_household_consumption', 
    'real_government_consumption',
    'real_capitalformation',
    'real_exports',
    'real_imports',
    'wages',
    'euribor',
    'gdp_deflator_growth_ea'
]

# Validate that all selected variables exist
missing_vars = [var for var in selected_variables if var not in variable_names]
if missing_vars:
    raise ValueError(f" The following selected variables are not available: {missing_vars}")

# Get indices for selected variables
plot_data = []
for var_name in selected_variables:
    var_idx = variable_names.index(var_name)
    plot_data.append((var_name, var_idx))

# Calculate optimal subplot layout (prefer more columns than rows for better readability)
n_vars = len(plot_data)
if n_vars <= 3:
    rows, cols = 1, n_vars
elif n_vars <= 6:
    rows, cols = 2, 3
elif n_vars <= 9:
    rows, cols = 3, 3
elif n_vars <= 12:
    rows, cols = 3, 4
else:
    rows, cols = 4, 4  # Max 4x4 grid, adjust as needed

# Create subplot titles
subplot_titles = [var.replace('_', ' ').title() for var, _ in plot_data]

# Create subplot figure
fig = make_subplots(
    rows=rows, 
    cols=cols,
    subplot_titles=subplot_titles,
    vertical_spacing=0.15,
    horizontal_spacing=0.06
)

# Add traces for each selected variable
for i, (var_name, var_idx) in enumerate(plot_data):
    row = (i // cols) + 1
    col = (i % cols) + 1
    
    series_data = results.get_scalar_timeseries()[var_name]
    fig.add_trace(
        go.Scatter(
            x=time_quarters,
            y=series_data,
            mode='lines',
            name=var_name,
            line=dict(width=2.5),
            hovertemplate='<b>%{fullData.name}</b><br>' +
                         'Quarter: %{x}<br>' +
                         'Value: %{y:.4f}<br>' +
                         '<extra></extra>'
        ),
        row=row, col=col
    )

# Update layout
fig.update_layout(
    title={
        'text': f'BeforeIT Model: Economic Indicators (Baseline Simulation)<br><sub>Showing {n_vars} selected variables over {T} quarters</sub>',
        'x': 0.5,
        'font': {'size': 16}
    },
    height=200 * rows + 100,  # Dynamic height based on number of rows
    width=1200,
    showlegend=False,  # Turn off legend since subplot titles show variable names
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Update axes labels
fig.update_xaxes(title_text="Quarters", showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the plot
fig.show()


## 4. Test Alternate Specs

Now let's test how changing parameters affects the model outcomes. We'll modify a few key parameters and compare the results.


In [32]:
# Let's test calibration by changing some key parameters
# Select a few important parameters to modif
key_params_to_test = ['psi', 'theta_UB', 'tau_INC']  # consumption propensity, unemployment benefits, income tax

# Find which of our target parameters are actually in the calibratable set
test_params = []
test_indices = []

for param in key_params_to_test:
    if param in param_names:
        idx = param_names.index(param)
        test_params.append(param)
        test_indices.append(idx)
        print(f"  Test parameter: {param} (current value: {current_values[param]:.4f})")
    else:
        print(f"  ⚠️  Parameter {param} not found in calibratable parameters")

if not test_params:
    # If none of our preferred parameters are available, use the first few
    test_params = param_names[:3]
    test_indices = list(range(3))
    print(f"  Using first 3 available parameters: {test_params}")

print("\nCurrent parameter values:")
for param in test_params:
    print(f"  • {param:12s}: {current_values[param]:.4f}")


  Test parameter: psi (current value: 0.9097)
  Test parameter: theta_UB (current value: 0.3586)
  Test parameter: tau_INC (current value: 0.2134)

Current parameter values:
  • psi         : 0.9097
  • theta_UB    : 0.3586
  • tau_INC     : 0.2134


In [33]:
# Create a modified parameter set
modified_params = baseline_params.copy()

# Increase the first test parameter by 10%
if test_indices:
    original_value = modified_params[test_indices[0]]
    modified_params[test_indices[0]] = original_value * 1.1
    
    print(f"Modified {test_params[0]}: {original_value:.4f} → {modified_params[test_indices[0]]:.4f} (+10%)")
    
    # Run simulation with modified parameters
    print("Running modified simulation...")
    modified_results = model.run_simulation(
        parameters=modified_params,
        T=T,
        seed=seed,
        ensemble_size=4
    )
    
    print("Modified simulation completed!")
else:
    print("No parameters available for testing")
    modified_results = results  # Use baseline results as fallback


INFO:uk_abm.julia_model_wrapper:Running simulation with T=40, ensemble_size=4, seed=42
INFO:uk_abm.julia_model_wrapper:Simulation completed successfully


Modified psi: 0.9097 → 1.0006 (+10%)
Running modified simulation...
Modified simulation completed!


In [36]:
# Compare baseline vs modified results using the same plotly structure
param_name = test_params[0] if test_params else "parameter"

# Create comparison figure with the same layout as the original
comparison_fig = make_subplots(
    rows=rows, 
    cols=cols,
    subplot_titles=subplot_titles,
    vertical_spacing=0.15,
    horizontal_spacing=0.06
)

# Add traces for both baseline and modified results
for i, (var_name, var_idx) in enumerate(plot_data):
    row = (i // cols) + 1
    col = (i % cols) + 1
    
    # Add baseline trace
    comparison_fig.add_trace(
        go.Scatter(
            x=time_quarters,
            y=results.get_scalar_timeseries()[var_name],
            mode='lines',
            name='Baseline',
            line=dict(width=2.5, color='blue'),
            hovertemplate='<b>Baseline - %{fullData.name}</b><br>' +
                         'Quarter: %{x}<br>' +
                         'Value: %{y:.4f}<br>' +
                         '<extra></extra>',
            showlegend=(i == 0)  # Only show legend for first subplot
        ),
        row=row, col=col
    )
    
    # Add modified trace
    comparison_fig.add_trace(
        go.Scatter(
            x=time_quarters,
            y=modified_results.get_scalar_timeseries()[var_name],
            mode='lines',
            name='Modified',
            line=dict(width=2.5, color='red', dash='dash'),
            hovertemplate='<b>Modified - %{fullData.name}</b><br>' +
                         'Quarter: %{x}<br>' +
                         'Value: %{y:.4f}<br>' +
                         '<extra></extra>',
            showlegend=(i == 0)  # Only show legend for first subplot
        ),
        row=row, col=col
    )

# Update layout
comparison_fig.update_layout(
    title={
        'text': f'Calibration Test: Effect of Increasing {param_name} by 10%<br><sub>Baseline vs Modified simulation over {T} quarters</sub>',
        'x': 0.5,
        'font': {'size': 16}
    },
    height=200 * rows + 100,  # Same dynamic height as original
    width=1200,
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update axes labels
comparison_fig.update_xaxes(title_text="Quarters", showgrid=True, gridwidth=1, gridcolor='lightgray')
comparison_fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the comparison plot
comparison_fig.show()
